In [2]:
import pandas as pd
import numpy as np

In [25]:
train = pd.read_csv('train.tsv', sep = '\t')
dev = pd.read_csv('dev.tsv', sep = '\t')
test = pd.read_csv('test.tsv', sep = '\t')

In [14]:
train['claim'][0]

'"The money the Clinton Foundation took from from foreign governments while Hillary Clinton was secretary of state ""is clearly illegal. … The Constitution says you can’t take this stuff."'

In [16]:
train['main_text'][0]

'"Hillary Clinton is in the political crosshairs as the author of a new book alleges improper financial ties between her public and personal life. At issue in conservative author Peter Schweizer’s forthcoming book Clinton Cash are donations from foreign governments to the Clinton Foundation during the four years she served as secretary of state. George Stephanopoulos used an interview with Schweizer on ABC This Week to point out what other nonpartisan journalists have found: There is no ""smoking gun"" showing that donations to the foundation influenced her foreign policy decisions. Still, former Republican House Speaker Newt Gingrich says the donations are ""clearly illegal"" under federal law. In his view, a donation by a foreign government to the Clinton Foundation while Clinton was secretary of state is the same as money sent directly to her, he said, even though she did not join the foundation’s board until she left her post. ""The Constitution of the United States says you cannot

In [18]:
train['explanation'][0]

'"Gingrich said the Clinton Foundation ""took money from from foreign governments while (Hillary Clinton) was secretary of state. It is clearly illegal. … The Constitution says you can’t take this stuff."" A clause in the Constitution does prohibit U.S. officials such as former Secretary of State Hillary Clinton from receiving gifts, or emoluments, from foreign governments. But the gifts in this case were donations from foreign governments that went to the Clinton Foundation, not Hillary Clinton. She was not part of the foundation her husband founded while she was secretary of state. Does that violate the Constitution? Some libertarian-minded constitutional law experts say it very well could. Others are skeptical. What’s clear is there is room for ambiguity, and the donations are anything but ""clearly illegal."" The reality is this a hazy part of U.S. constitutional\xa0law. '

In [22]:
test.label.value_counts().index

Index(['true', 'false', 'mixture', 'unproven'], dtype='object')

In [26]:
# Take only the 4 important labels

labels = ['true', 'false', 'mixture', 'unproven']
train = train[train.label.isin(labels)]
dev = dev[dev.label.isin(labels)]

In [29]:
dev.label.value_counts()

true        629
false       380
mixture     164
unproven     41
Name: label, dtype: int64